In [347]:
%load_ext autoreload
%autoreload 2

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [392]:
from data_preprocessing.tg_json_reader import DataRetriever
from data_preprocessing.conversation_builder import create_conversations
from data_preprocessing.util import write_txt, read_txt, train_bpe_tokenizer
from data_preprocessing.dataset import ChatDataset

from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from tokenizers.processors import BertProcessing

In [364]:
path = "/Users/diat.lov/Downloads/Telegram Desktop/DataExport_2021-11-05/result.json"
data_retriever = DataRetriever(path)
dictinoary = data_retriever.get_message_history_by_name("Катя")

In [365]:
df = data_retriever.get_pandas_df(dictinoary)

In [366]:
df = data_retriever.tg_specific_preprocessing(df)

In [395]:
all_conversations = create_conversations(df, return_str=False)

In [396]:
all_conversations[0]

'<Kate> Пхахахаххахахаха <Kate> Ты когда домой? <Daria> В 8 <Kate> Что там по зп? <Daria> Ты спишь?'

In [371]:
write_txt(all_conversations, "/Users/diat.lov/GitHub/tg_lm_chatbot/data/conversations.txt")

In [ ]:
train_bpe_tokenizer("/Users/diat.lov/GitHub/tg_lm_chatbot/data/conversations.txt",
                    "/Users/diat.lov/GitHub/tg_lm_chatbot/data/tokenizer")

In [390]:
tokenizer = GPT2Tokenizer.from_pretrained("/Users/diat.lov/GitHub/tg_lm_chatbot/data/tokenizer")

In [397]:
dataset = ChatDataset(all_conversations)

In [400]:
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]